# 加载预训练的Inception-v3 Model

In [ ]:
from keras.applications import inception_v3
from keras import backend as K

# 不需要训练模型 这个命令会关闭Keras的训练模式
K.learning_phase(0)

# 构建不包含全连接层的Inception-v3模型 使用预训练的ImageNet权重
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

## 计算损失 梯度上升过程中需要最大化的量 将多个层的所有过滤器的激活最大化
# 设置DeepDream配置


In [ ]:
# 这个字典将层的名称映射为一个系数 这些系数控制每一层对最终贡献的损失大小
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

# 定义需要最大化的损失函数


In [ ]:
layer_dict = dict([layer.name, layer] for layer in model.layers)) # 将层的名称映射到层实例
loss = K.variable(0.) # 初始化损失为0 定义损失的时候将对层的贡献添加到标量里面

for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output # 获取层的输出
    scaling = K.prod(K.cast(K.shape(activation), 'float32')) # 获取激活的维度乘积
    loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling # 计算损失

# 定义梯度上升过程


In [ ]:
dream = model.input() # 这个张量用来保存生成的梦境图像
grads = K.gradients(loss, dream)[0]